# Combining Semantic and LLM routing

In [2]:
# Import necessary libraries
import boto3
import json
from dotenv import load_dotenv, find_dotenv
import os


# loading environment variables that are stored in local file dev.env
local_env_filename = 'bedrock-router-eval.env'
load_dotenv(find_dotenv(local_env_filename),override=True)

os.environ['REGION'] = os.getenv('REGION')
REGION = os.environ['REGION']

In [3]:
# Step 1: Define categories and difficulties
topics = ['code', 'summarization', 'general']
difficulty_levels = ['easy', 'medium', 'hard']
synthetic_data_model = "anthropic.claude-3-haiku-20240307-v1:0"

In [4]:
brt = boto3.client(service_name='bedrock-runtime', region_name=REGION)
data_gen_prompt = f"generate a json file with example prompts across the following topics {topics} and following category level {difficulty_levels}. Your answer will only contain the json formatted answer."
from botocore.exceptions import ClientError

def syntheticdatagen(topics,difficulty_levels):
    native_request = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 512,
        "temperature": 0.5,
        "messages": [
            {
                "role": "user",
                "content": [{"type": "text", "text": data_gen_prompt}],
            }
        ],
    }
    # Convert the native request to JSON.
    request = json.dumps(native_request)
    
    try:
        # Invoke the model with the request.
        response = brt.invoke_model(modelId=synthetic_data_model, body=request)
    
    except (ClientError, Exception) as e:
        print(f"ERROR: Can't invoke '{synthetic_data_model}'. Reason: {e}")
        exit(1)
    
    # Decode the response body.
    model_response = json.loads(response["body"].read())
    
    # Extract and print the response text.
    response_text = model_response["content"][0]["text"]
    print(data_gen_prompt)
    return response_text

In [5]:
syntheticdatagen(topics,difficulty_levels)

generate a json file with example prompts across the following topics ['code', 'summarization', 'general'] and following category level ['easy', 'medium', 'hard']. Your answer will only contain the json formatted answer.


'{\n  "code": {\n    "easy": [\n      "Write a function that takes two numbers as input and returns their sum.",\n      "Create a simple program that prints the first 10 numbers of the Fibonacci sequence.",\n      "Write a script that checks if a given string is a palindrome."\n    ],\n    "medium": [\n      "Implement a binary search algorithm to find a target value in a sorted array.",\n      "Write a function that takes a list of numbers and returns the second-largest number.",\n      "Create a program that simulates a simple calculator with basic arithmetic operations."\n    ],\n    "hard": [\n      "Develop a program that solves a Sudoku puzzle given a partially filled grid.",\n      "Implement a recursive function to find the nth Fibonacci number.",\n      "Write a program that finds the longest common subsequence between two given strings."\n    ]\n  },\n  "summarization": {\n    "easy": [\n      "Summarize the main points of a short news article in a few sentences.",\n      "Pr